In [17]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import imageio.v2 as imageio




num_substrate = 20
num_enzyme = 10
num_trace = 100

num_time_step = 250
simulation_time = 2.5

dt = simulation_time / num_time_step
q1 = num_enzyme

c1 = 1.2
c2 = 0.3
c3 = 2.9

v1 = [-1, -1]
v2 = [1, 1]
v3 = [0, 1]

a1 = lambda enzyme, substrate: enzyme * substrate * c1
a2 = lambda enzyme: (q1 - enzyme) * c2
a3 = lambda enzyme: (q1 - enzyme) * c3

trace_time = [0 for _ in range(num_trace)]
trace_substrate_state = [num_substrate for _ in range(num_trace)]
trace_enzyme_state = [num_enzyme for _ in range(num_trace)]


def calc_p_from_trace(trace_substrate_state, trace_enzyme_state, num_substrate, num_enzyme, tSteps):
    P = pd.DataFrame([[0 for _ in range(num_substrate)] for _ in range(num_enzyme)])
    
    #There was a problem with the for loop so I
    #changed it and I'm not sure if it's the right way to do it but it works
    for t in range(len(trace_substrate_state)):
        substrate_idx = min(trace_substrate_state[t], num_substrate - 1)# this ensures that the index is within the bounds of the DataFrame's columns 
        enzyme_idx = min(trace_enzyme_state[t], num_enzyme - 1) # this ensures that the index is within the bounds of the DataFrame's rows
        P.iloc[enzyme_idx, substrate_idx] += 1
     
    
    P = P / num_trace
    sns.heatmap(P, annot=True)
    plt.xlabel('Number of Substrates') 
    plt.ylabel('Number of Enzyme')
    plt.savefig(f"Gillespie_traces/Gillespie_trace{tSteps}.png")
    print(f"Image saved: Gillespie_traces/Gillespie_trace{tSteps}.png")
    
    plt.close()
    
    return P


for tSteps in range(num_time_step): 
    for t in range(num_trace):
        enzyme, substrate = trace_enzyme_state[t], trace_substrate_state[t]
        
        while trace_time[t] < dt * tSteps:
            a1X = a1(substrate, enzyme)
            a2X = a2(enzyme)
            a3X = a3(enzyme)
            
            rand_uniform = np.random.uniform(low=0.0, high=1.0, size=1)# why we generate a random number!?
            xi1 = rand_uniform * (a1X + a2X + a3X)
            
            if (substrate, enzyme) == (0, 10):
                trace_time[t] = simulation_time
            elif xi1 < a1X:
                substrate, enzyme = substrate + v1[0], enzyme + v1[1]
            elif xi1 < a1X + a2X:
                substrate, enzyme = substrate + v2[0], enzyme + v2[1]
            else:
                substrate, enzyme = substrate + v3[0], enzyme + v3[1]
            
            try:
                xi2 = -math.log(1 - rand_uniform) / (a1X + a2X + a3X)# why this calculation?!
            except ZeroDivisionError:
                xi2 = 10000
            
            trace_time[t] += xi2
        
        trace_substrate_state[t] = substrate
        trace_enzyme_state[t] = enzyme

P = calc_p_from_trace(trace_substrate_state, trace_enzyme_state, num_substrate, num_enzyme, tSteps)
with imageio.get_writer('Gillespie_gif.gif', mode='I') as writer:
    for filename in range(num_time_step):
        image_path = os.path.join('Gillespie_traces', f'Gillespie_trace{filename}.png')
        image = imageio.imread(image_path)
        writer.append_data(image)  
        
        

"""
I tried to simulate, but the folder that is used to store
images has a problem, which unfortunately could not be solved.
"""




Image saved: Gillespie_traces/Gillespie_trace249.png


FileNotFoundError: No such file: 'C:\Users\saman\Gillespie_traces\Gillespie_trace0.png'